<div class='status' style='background-color: #0c4a6e; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>Appraisals - Analysing Pre-appraisal questions</b> - Integration with Notion <code>03</code></div>

1. **Notion Database Data Loader** - Custom Crewai Tool to load all information form a Notion Database including a list of page ids.
2. **Notion Page Data Loader** - Tool to load all the json information from a Notion database page, including creator information and comments.

<img src="diagram_01.png" width="550">

<div class='status' style='background-color: #f59e0b; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> Libraries</div>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import json
import yaml
from crewai import Agent, Task, Crew

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Loading <b>Agents + Tasks</b> form YAML file</div>

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Create <b>Custom Tools</b></div>

**Specify Notion Database ID** for report.  
Moving forward this should be passed to the crew as an input paramter.

Appraisals Database:  12dfdfd68a97808889d4d8c041a281eb

In [4]:
from crewai_tools import BaseTool
from notion_api.notionhelper import *
import requests
from typing import ClassVar, Union

class DatabaseDataFetcherTool(BaseTool):
    name: str = "Notion Database Data Fetcher"
    description: str = "Fetches Notion Database structure and properties."

    nh: ClassVar[NotionHelper] = NotionHelper()
    database_id: str = "12ffdfd68a978153b9acc9d276775977"
    
    def _run(self) -> Union[dict, str]:
        """
        Fetch all the properties and structure of a Notion database.
        """
        response = self.nh.get_database(database_id=self.database_id)
        return response
        
        if isinstance(response, dict):
            return response
        else:
            # Fallback in case of timeouts or other issues
            return {"error": "Failed to fetch data from Notion. Please try again."}


class PageDataFetcherTool(BaseTool):
    name: str = "Notion Database Pages Data Fetcher"
    description: str = "Fetches all the page content from a Notion database."

    nh: ClassVar[NotionHelper] = NotionHelper()
    database_id: str = "12ffdfd68a978153b9acc9d276775977"
    
    def _run(self) -> Union[list, str]:
        """
        Fetch all the pages properties in a Notion Database.
        """
        response = self.nh.get_all_pages_as_json(self.database_id)

        if isinstance(response, list):
            return response
        else:
            # Fallback in case of timeouts or other issues
            return {"error": "Failed to fetch data from Notion. Please try again."}

<div class='status' style='background-color: #0e7490; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Create <b>Crew, Agents and Tasks</b></div>

In [5]:
from crewai import LLM

#llm = LLM(model="llama3.1:latest", base_url="http://localhost:11434/v1", api_key="Ollama")
   


# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[DatabaseDataFetcherTool(), PageDataFetcherTool()],

)

analysis_agent = Agent(
  config=agents_config['analysis_agent'],
    # llm=llm,
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)

<div class='status' style='background-color: #4d7c0f; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Train</b> the Crew</div>

In [6]:
# crew.train(n_iterations=1, filename='training.pkl')

In [7]:
#crew.test(3)

<div class='status' style='background-color: #be123c; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Kickoff</b> the Crew</div>

In [8]:
# Kick off the crew and execute the process
result = crew.kickoff()

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Notion Databse Data Fetcher tool to gather data from the Notion database, this explains the structure of the database.



# Agent: Data Collection Specialist
## Thought: I need to gather information about the project's structure and features from the Notion database to create an initial understanding.
## Using tool: Notion Database Data Fetcher
## Tool Input: 
"{\"name\": \"Project Overview\", \"description\": \"\", \"args_schema\": \"dict\", \"return_direct\": false, \"verbose\": true}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Data Collection Specialist
## Thought: Thought: I need to properly format the input for the Notion Database Data Fetcher to gather the database structure and properties.
## Using tool: Notion Database Data Fetcher
## Tool Input: 
"{\"name\": \"Project Database\", \"desc

### **Cost** Estimation

In [9]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0051


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,33719,28119,5600,14


<div class='status' style='background-color: #0f766e; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'>Crewai<b> Final Output</b></div>

In [10]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

# Employee Appraisal Report: Appraisal Year 25/26

## Overview
This report summarizes the analysis of pre-appraisal feedback collected from employees to identify their accomplishments, goals, challenges, strengths, weaknesses, and training needs. The objective is to develop tailored Personal Development Plans (PDPs) and recommend topics for discussion during appraisal interviews.

## 1. Employee Accomplishments (Previous Year)
- **Successful Project Completion:**
  - Delivery of critical initiatives and adaptation to clinical protocol changes.
  - Effective management of patient care during high-pressure periods, notably during flu season.
  
- **Professional Development:**
  - Participation in training programs, enhancing clinical skills, including:
    - Advanced Care Planning
    - Emergency Response Certifications
    
- **Team Collaboration:**
  - Notable improvement in teamwork across interdisciplinary teams, resulting in:
    - Better patient outcomes
    - Increased workflow efficiency

## 2. Employee Goals and Objectives (Next Appraisal Period)
- **Enhancing Clinical Skills:**
  - Pursue additional training in:
    - Diabetes management
    - Mental health first aid
    - Advanced diagnostic techniques
    
- **Operational Improvements:**
  - Develop better time management strategies.
  - Improve patient scheduling methods to reduce wait times.

- **Leadership Development:**
  - Seek leadership roles and mentorship positions to nurture team dynamics and professional growth.

## 3. Identified Challenges
- **Workload Management:**
  - Difficulty balancing responsibilities during peak times, leading to potential burnout.

- **Digital Literacy:**
  - Struggles with adapting to new technologies, resulting in underutilization of tools.

- **Support and Resources:**
  - A need for enhanced training and resources to effectively tackle identified challenges.

## 4. Strengths, Weaknesses, and Opportunities
- **Strengths:**
  - Strong clinical knowledge and dedication to patient care.
  - Ability to work collaboratively within teams.
  - Openness to learning and adapting in fast-paced environments.

- **Weaknesses:**
  - Stress management issues and adaptability to rapid technological changes affecting workflows.

- **Opportunities:**
  - Streamlining workflows through improved resource allocation.
  - Enhancing digital literacy training.
  - Expanding professional development programs.

## 5. Training and Skill Development Needs
- **Identified Training Programs:**
  - Workshops on:
    - Time management
    - Use of digital tools
    - Stress management techniques
    - Advanced clinical training on emerging medical trends

- **Supportive Resources:**
  - Leadership to facilitate regular check-ins and ongoing feedback sessions for growth alignment.

## 6. Suggested Discussion Topics for Appraisal Interview
1. **Professional Development Aspirations:**
   - Discuss specific training programs and how they align with career advancement.
   
2. **Challenges and Support Required:**
   - Explore workload management strategies and the required resources/training for overcoming obstacles.

3. **Leadership and Mentorship Opportunities:**
   - Evaluate the potential for employees to assume leadership/mentorship roles and the support needed to succeed.

## 7. Customized Personal Development Plans (PDP)
### Employee 1: [Employee Name]
**Goal 1: Enhance clinical skills in diabetes management**
- **Specific:** Enroll in a certified diabetes management course.  
- **Measurable:** Complete the course with a passing grade by [Month/Year].  
- **Achievable:** Allocate 3 hours a week for study.  
- **Relevant:** Aligns with employee's aspiration for improved care quality.  
- **Time-bound:** Complete within 6 months; review progress monthly.

**Goal 2: Improve time management skills**
- **Specific:** Attend a time management workshop.  
- **Measurable:** Implement at least three strategies post-workshop and track effectiveness.  
- **Achievable:** Engage in practical exercises during the workshop.  
- **Relevant:** Directly ties to objectives of operational improvement.  
- **Time-bound:** Complete within 3 months, with follow-up reviews bi-monthly.

### Employee 2: [Employee Name]
**Goal 1: Develop leadership and mentorship skills**
- **Specific:** Participate in a leadership development program.  
- **Measurable:** Lead at least two team projects by the end of the appraisal period.  
- **Achievable:** Utilize skills learned from the program in real scenarios.  
- **Relevant:** Supports the employee's goal of taking on leadership roles.  
- **Time-bound:** Complete program in 4 months, with a review at the next appraisal.

**Goal 2: Increase digital literacy**
- **Specific:** Attend training for new digital tools.  
- **Measurable:** Achieve proficiency in at least three tools by [Month/Year].  
- **Achievable:** Dedicate 2 hours per week to training sessions.  
- **Relevant:** Addresses identified challenges in adapting to new technology.  
- **Time-bound:** Complete training within 5 months, with progress assessments every month.

## Conclusion
This comprehensive report acts as a framework for the upcoming appraisal discussions, focusing on employee development aligned with organizational goals. The tailored PDPs are designed to foster continuous improvement and support employees in navigating their challenges while enhancing their contributions to the workplace. Regular reviews and adaptations will be essential to ensure the effectiveness of these plans, creating an environment of ongoing growth and achievement.

<img src="../3.png" align="right" width=300>